## Read in a CSV and write out python code defining a data class for each row

In [50]:
import csv

def python_from_csv(filename, class_name):
    source_file = f"..\\django\\QuadEBdb\\tom\\generated_code\\{class_name}.py"
    indent = "    "

    header, first_row = list(csv.reader(open(filename)))[:2]

    with open(source_file, "w") as f:
        f.write("from django.db import models\n")
        f.write("\n")
        f.write(f"class {class_name}(models.Model):\n")
        for column_name, example_data in zip(header, first_row):
            try:
                _ = float(example_data)
                if example_data.count(".") > 0:
                    field_type = "FloatField(null=True)"
                else:
                    if len(example_data) > 8:
                        field_type = "BigIntegerField(null=True)"
                    else:
                        field_type = "IntegerField(null=True)"
            except:
                field_type = 'CharField(max_length=100, null=True)'
            if column_name.lower() == "id":
                #django creates a column named id as the primary key, so we can't also use it here
                column_name = "Identifier"
            f.write(f"{indent}{column_name.replace(" ", "_")} = models.{field_type}\n")

# python_from_csv("TICv8 Targets.csv", "Tess_TICv8")
# python_from_csv("GAIA DR2 Example.csv", "Gaia_DR2")
# python_from_csv("GAIA Neighbors.csv", "Gaia_DR3")
python_from_csv("2MASS.csv", "TwoMass")


In [7]:
# 2024-10-10 do a quickie look to find TICs that might be good for light curves w/ the TV-85

from functools import partial
from sqlite3 import connect
import sys

from astropy.table import Table

sys.path.append("../../")
import astropaul.targetlistcreator as tlc

conn = connect("../../django/TargetDB/db.sqlite3")

creator = tlc.TargetListCreator(connection=conn)
creator.steps = [
    tlc.add_targets,
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="tess"),
    tlc.add_lists,
]

targets = Table.from_pandas(creator.calculate())

#TODO: add a function to targetlistcreator that takes an astroplan observer and (start, end) time and outputs observability


In [6]:
targets

target_name,target_source,target_type,ra,dec,pmra,pmdec,parallax,Vmag,Teff,list_Featured_targets,list_Gemini_2024B_Proposal,list_Gemini_2025A_Proposal,list_HQND,list_Kostov_2022,list_Kostov_2023,list_LBT_2023-12-06,list_LBT_2023B,list_Speckle_Resolved
str18,str28,str18,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bool,bool,bool,bool,bool,bool
TIC 270360534,Kostov 2023 arXiv:2309.14200,QuadEB,263.430502039255,43.860752149892,-11.9616,-23.9874,1.74434,12.041,5048.88,False,False,False,False,False,True,False,False,False
TIC 219469945,Kostov 2022 arXiv:2202.05790,QuadEB,241.047907546962,43.0303006747434,-0.866116,-13.0913,1.08328,12.507,6322.79,False,False,False,False,True,False,False,True,False
TIC 20212631,Kostov 2023 arXiv:2309.14200,QuadEB,227.038060401645,39.97024000055,-82.8253,60.7128,5.36793,10.58,5665.4,False,False,False,False,False,True,False,False,False
TIC 150055835,Kostov 2023 arXiv:2309.14200,QuadEB,69.9253097979945,29.1345840372292,1.09215,-1.99413,0.347484,15.116,6445.0,False,True,True,False,False,True,False,False,False
TIC 161043618,Kostov 2022 arXiv:2202.05790,QuadEB,223.425163143389,52.7158479482434,2.17801,-3.59429,2.02761,12.497,5860.0,False,False,False,False,True,False,False,True,True
TIC 58539786,Kostov 2023 arXiv:2309.14200,QuadEB,65.8340211217007,29.1427494807598,5.51137,-5.3655,0.83413,15.266,5597.0,False,True,True,False,False,True,False,False,False
TIC 441794509,Kostov 2022 arXiv:2202.05790,QuadEB,263.598848681812,74.4722587998312,3.44165,-21.2733,1.10894,12.819,6221.0,False,False,False,False,True,False,True,True,False
TIC 219006972,Kostov 2023 arXiv:2309.14200,QuadEB,221.202871081398,66.3786726710139,6.64058,-3.44764,0.703547,14.312,5878.0,False,True,False,False,False,True,False,False,False
TIC 304713857,Kostov 2023 arXiv:2309.14200,QuadEB,304.203363762097,18.5761932727513,-13.8,-4.6,--,12.211,5866.0,False,False,False,False,False,True,False,False,False
